This IPYNB contains cells that process form feedback data.

Author: Vamsy Malladi


# Import Statements

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy as sp
import os
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from nltk.corpus import wordnet
from translate import Translator
from matplotlib.lines import Line2D

In [8]:
# Reading all the SCL datasets

df_list = list()
df_list_baseline = list()
df_list_ssst = list()
df_list_relaxation = list()
file_names = glob.glob("SCL_Data/*.csv")
for file_name in file_names:
    df = pd.read_csv(file_name, sep = ";")
    id = file_name.split('\\')[1].split('_')[0]
    df['ID'] = id
    df['ID'] = df['ID'].astype('str').astype('int')
    df_list.append(df)
    pd.to_datetime(df['Interpolated Timestamp (256Hz)'], infer_datetime_format=True)


ParserError: Unknown string format: 2021.09.23 11:58.54.127

# Data Preprocessing
Goal is to get the data that is relevant to the analysis (415 seconds of SSST, 5 minutes of Relaxation phase and 2 minutes of baseline)

In [3]:
# Add the Phase column to each dataframe.

for i, df in enumerate(df_list):
    df.columns = df.columns.str.lstrip()
    df['Phase'] = np.select([(df['ID'] <= 6),
                            (df['ID'] > 6) & (df['ID'] <= 14) | (df['ID'] == 31) | (df['ID'] == 32),
                            (df['ID'] > 14) & (df['ID'] <= 22) | (df['ID'] == 33),
                            (df['ID'] > 22) & (df['ID'] <= 30) | (df['ID'] == 34) | (df['ID'] == 35) | (df['ID'] == 36)],
                            ["SSST", "1", "2", "3"])
    df.drop(df[df['ID'].isin([4, 5, 10, 12, 20, 24, 26, 27])].index, inplace=True)
    # df['Task'] = np.select()


In [5]:
df.dtypes

Interpolated Timestamp (256Hz)     object
Data                              float64
Systemtime                         object
'Missing' Samples                   int64
Flags                              object
ID                                  int32
Phase                              object
dtype: object

In [6]:
df.head(20)

,Interpolated Timestamp (256Hz),Data,Systemtime,'Missing' Samples,Flags,ID,Phase
0,2022.04.22 10:11.04.952,2.896654,\t2022.04.22 10:11.05.003,-1,\t,36,3
1,2022.04.22 10:11.04.956,2.901623,\t2022.04.22 10:11.05.003,-1,\t,36,3
2,2022.04.22 10:11.04.960,2.901623,\t2022.04.22 10:11.05.003,-1,\t,36,3
3,2022.04.22 10:11.04.963,2.896654,\t2022.04.22 10:11.05.003,-1,\t,36,3
4,2022.04.22 10:11.04.967,2.901623,\t2022.04.22 10:11.05.003,-1,\t,36,3
5,2022.04.22 10:11.04.971,2.896654,\t2022.04.22 10:11.05.003,-1,\t,36,3
6,2022.04.22 10:11.04.975,2.891686,\t2022.04.22 10:11.05.003,-1,\t,36,3
7,2022.04.22 10:11.04.979,2.896654,\t2022.04.22 10:11.05.003,-1,\t,36,3
8,2022.04.22 10:11.04.983,2.896654,\t2022.04.22 10:11.05.003,-1,\t,36,3
9,2022.04.22 10:11.04.987,2.896654,\t2022.04.22 10:11.05.003,-1,\t,36,3
